[toc]

# Tensorflow shape

## 三种形状操作

获取变量维度是一个使用频繁的操作，在tensorflow中获取变量维度主要用到的操作有以下三种：

*   Tensor.shape
*   Tensor.get_shape()
*   tf.shape(input, name=None, out_type=tf.int32)

对上面三种操作做一下简单分析：（这三种操作先记作**A、B、C**）

*   **A** 和 **B** 基本一样，只不过前者是Tensor的属性变量，后者是Tensor的函数。
*   **A** 和 **B** 均返回TensorShape类型，而 **C** 返回一个 1D 的 out_type 类型的Tensor。
*   **A** 和 **B** 可以在任意位置使用，而 **C** 必须在Session中使用。
*   **A** 和 **B** 获取的是静态shape，**可以返回不完整的shape**； **C** 获取的是动态的shape，必须是完整的shape。

另外，补充从TenaorShape变量中获取具体维度数值的方法

## 静态shape

In [1]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [None, None])

# 直接获取TensorShape变量的第i个维度值

print(x.shape)
print(x.shape[0].value)
print(x.get_shape()[1].value)

# 将TensorShape变量转化为list类型，然后直接按照索引取值
print(x.get_shape().as_list())

(?, ?)
None
None
[None, None]


下面给出全部的示例程序：

In [2]:
import tensorflow as tf

x1 = tf.constant([[1,2,3],[4,5,6]])

# 占位符创建变量，第一个维度初始化为None，表示暂不指定维度
x2 = tf.placeholder(tf.float32,[None, 2,3])

print('x1.shape:',x1.shape)
print('x2.shape:',x2.shape)
print('x2.shape[1].value:',x2.shape[1].value)
print('tf.shape(x1):',tf.shape(x1))
print('tf.shape(x2):',tf.shape(x2))
print('x1.get_shape():',x1.get_shape())
print('x2.get_shape():',x2.get_shape())
print('x2.get_shape().as_list[1]:',x2.get_shape().as_list()[1])

shapeOP1 = tf.shape(x1)
shapeOP2 = tf.shape(x2)
with tf.Session() as sess:
    print('Within session, tf.shape(x1):',sess.run(shapeOP1))
    # 由于x2未进行完整的变量填充，其维度不完整，因此执行下面的命令将会报错
    # print('Within session, tf.shape(x2):',sess.run(shapeOP2)) # 此命令将会报错

x1.shape: (2, 3)
x2.shape: (?, 2, 3)
x2.shape[1].value: 2
tf.shape(x1): Tensor("Shape:0", shape=(2,), dtype=int32)
tf.shape(x2): Tensor("Shape_1:0", shape=(3,), dtype=int32)
x1.get_shape(): (2, 3)
x2.get_shape(): (?, 2, 3)
x2.get_shape().as_list[1]: 2
Within session, tf.shape(x1): [2 3]


### 动态shape

In [13]:
import tensorflow as tf
tf.reset_default_graph()

a = tf.placeholder("float32", [None])  # 长度未知
b = tf.placeholder("float32", [None])  # 长度未知


# 用 `tf.shape(x)[0]` 而不是 `x.shape[0]`
n = tf.shape(a)[0] # Tensor("strided_slice:0", shape=(), dtype=int32)
m = tf.shape(b)[0] # Tensor("strided_slice_1:0", shape=(), dtype=int32)

c = tf.zeros([n, m])  # 用动态长度指定形状
c_shape = tf.shape(c)

with tf.Session() as sess:
    feed_dict = {a: [1, 2, 3], b: [4, 5]}
    print(sess.run(c_shape, feed_dict=feed_dict))

Tensor("strided_slice:0", shape=(), dtype=int32)
Tensor("strided_slice_1:0", shape=(), dtype=int32)
[3 2]


注意，在上面的示例中，n 和 m 都是 Tensor 对象。

如果不使用 `tf.shape` 而是用 `x.shape` 下面的语句会报错，报错的是 `c = tf.zeros([n, m])`，因为 n 和 m 此时都是 `None`，创建一个形状为 [None, None] 的全零向量就会报错。

In [7]:
import tensorflow as tf

a = tf.placeholder("float32", [None])  # 长度未知
b = tf.placeholder("float32", [None])  # 长度未知

n = a.shape[0]
m = b.shape[0]

c = tf.zeros([n, m])
c_shape = tf.shape(c)

with tf.Session() as sess:
    feed_dict = {a: [1, 2, 3], b: [4, 5]}
    print(sess.run(c_shape, feed_dict=feed_dict))

TypeError: __int__ returned non-int (type NoneType)

## get_shape_list

由于静态 shape 和动态 shape 的特点，一个返回静态shape，当静态 shape没有指定时返回动态 shape 的函数就很有必要了。

**疑问**：如果动态shape可以做静态shape 不能做的事情，那么要静态shape有啥用？都要动态shape好了？

在 bert 的实现中，有个 [get_shape_list](https://github.com/google-research/bert/blob/eedf5716ce1268e56f0a50264a88cafad334ac61/modeling.py#L895:5) 函数就是这个功能


In [15]:
def get_shape_list(tensor, expected_rank=None, name=None):
    """Returns a list of the shape of tensor, preferring static dimensions.
  Args:
    tensor: A tf.Tensor object to find the shape of.
    expected_rank: (optional) int. The expected rank of `tensor`. If this is
      specified and the `tensor` has a different rank, and exception will be
      thrown.
    name: Optional name of the tensor for the error message.
  Returns:
    A list of dimensions of the shape of tensor. All static dimensions will
    be returned as python integers, and dynamic dimensions will be returned
    as tf.Tensor scalars.
  """
    if name is None:
        name = tensor.name

    if expected_rank is not None:
        assert_rank(tensor, expected_rank, name) # 判断是否是期望的形状

    shape = tensor.shape.as_list() # 静态shape

    non_static_indexes = []
    for (index, dim) in enumerate(shape):
        if dim is None: # 如果静态shape为None，那么就添加到 non_static_indexes 中
            non_static_indexes.append(index)

    if not non_static_indexes: # 全是静态shape
        return shape

    dyn_shape = tf.shape(tensor)
    for index in non_static_indexes:
        shape[index] = dyn_shape[index]
    return shape # 用动态shape填充那些是None的静态shape